In [1]:
import numpy as np
import pandas as pd
from bokeh.layouts import row, widgetbox, column
from bokeh.models import CustomJS, Slider, Select, HoverTool 
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import push_notebook, output_notebook, curdoc
from bokeh.client import push_session
from bokeh.embed import file_html
from bokeh.resources import CDN
#from bokeh.scatter_with_hover import scatter_with_hover
output_notebook()

Loading BokehJS ...

In [2]:
df_p = pd.read_csv('../../csvs/initial_orbital_elements.csv', index_col=0)
del df_p['instability_time']
del df_p['Rel_Eerr']
#del df['Rel_Eerr_short']
del df_p["runstring"]
#Nrows = df.shape[0]
#print Nrows, len(np.where(df['Stable']==0)[0])
print df_p.shape
df_p.head()
val_a = df_p.iloc[:500,:]#[:200,:] #[:50, :]

(25000, 29)


In [3]:
print val_a.head()

   Stable   RHill12   RHill23     beta12     beta23            m1   a1  \
0       0  0.026033  0.023222   8.508586  15.826967  3.414016e-05  1.0   
1       0  0.022704  0.031284   8.397002  15.367618  2.529838e-07  1.0   
2       0  0.036304  0.032149   3.461288  25.229467  7.380789e-05  1.0   
3       1  0.011704  0.014963  18.513681  19.145889  5.174804e-07  1.0   
4       0  0.006199  0.008634   5.494196   2.928938  5.106746e-07  1.0   

         P1        e1   pomega1    ...       Omega2        f2            m3  \
0  0.999983  0.005031  3.171041    ...     1.573978 -0.567081  1.826900e-06   
1  1.000000  0.010350 -3.564030    ...    -1.749616  1.437355  1.955797e-05   
2  0.999963  0.051912  0.995149    ...    -0.032550 -0.317958  1.477931e-07   
3  1.000000  0.000152 -3.328433    ...     0.946351 -2.294238  1.287851e-06   
4  1.000000  0.001457  1.738448    ...    -0.455684 -1.074927  1.542321e-06   

         a3        P3        e3   pomega3      inc3    Omega3        f3  
0  1.5

In [4]:
# Here is a dict of some keys that I want to be able to pick from for plotting
labels = list(df_p.columns.values)
axis_map = {key:key for key in labels}

#add color and legend
colormap = {1: 'green', 0: 'blue'}
label = {1:"Stable", 0:"Unstable"}
colors = [colormap[x] for x in val_a['Stable']]
labels = [label[x] for x in val_a["Stable"]]

#axis_map.pop("Stable", None)
print axis_map

{'f1': 'f1', 'f2': 'f2', 'f3': 'f3', 'Stable': 'Stable', 'beta23': 'beta23', 'e1': 'e1', 'RHill12': 'RHill12', 'm1': 'm1', 'Omega3': 'Omega3', 'm3': 'm3', 'm2': 'm2', 'e2': 'e2', 'P2': 'P2', 'P3': 'P3', 'P1': 'P1', 'pomega1': 'pomega1', 'pomega2': 'pomega2', 'pomega3': 'pomega3', 'a1': 'a1', 'a3': 'a3', 'a2': 'a2', 'RHill23': 'RHill23', 'Omega1': 'Omega1', 'e3': 'e3', 'Omega2': 'Omega2', 'inc1': 'inc1', 'inc2': 'inc2', 'inc3': 'inc3', 'beta12': 'beta12'}


In [5]:
#This is to update during the callback
#original example in working minimal example
code = ''' var data = source.data;

           //axis values with select widgets
           var value1 = val1.value;
           var value2 = val2.value;

           var original_data = original_source.data
           // get data corresponding to selection
           
           x = original_data[value1];
           y = original_data[value2];
          
           
           
           //console.log("Callback 2 runnning")

           
    
           
           // #need to create filter based on min and max vals
          
           var val1 = slide.value
           var val2 = slide2.value
           var val3 = slide3.value
           var val4 = slide4.value
           
           function inBetween2X(x) {
               return (val1 < x & x < val3)

           }
           
           function inBetween2Y(x) {
               return (val2 < x & x<val4)
       
           }
           //console.log("Create filter")
           source.trigger('change');
           filter_x = x.filter(inBetween2X);
           filter_y = y.filter(inBetween2Y);
                  
           data['x'] = filter_x;
           data['y'] = filter_y;
           
           
           
           
           

           
           '''

code2 = ''' var data = source.data;
            
           //axis values with select widgets
           var value1 = val1.value;
           var value2 = val2.value;
                     
           var original_data = original_source.data
           // get data corresponding to selection
           
           x = original_data[value1];
           y = original_data[value2];
           data['x'] = x;
           data['y'] = y;
           
           var start1 = Math.min.apply(Math, original_data[value1])
           var end1 = Math.max.apply(Math, original_data[value1])
           
           var start2 = Math.min.apply(Math, original_data[value2])
           var end2 = Math.max.apply(Math, original_data[value2])
           

           
           var val1 = slide.value
           var val2 = slide2.value
           var val3 = slide3.value
           var val4 = slide4.value
           
           stepsize = 20.
           
           //Need some way of mapping the current value to a step number
           //set the new value to that step number in the new range
           //////////////////////////////////////////
           old_start1 = slide.get("start")
           old_start2 = slide2.get("start")
           old_end1 = slide3.get("end")
           old_end2 = slide4.get("end")
           
           pos1 = Math.round((val1-old_start1)/(old_end1-old_start1)*stepsize)
           slide.set('value', start1 + pos1 * (end1-start1)/stepsize)
           
           pos2 = Math.round((val2-old_start2)/(old_end2-old_start2)*stepsize)
           slide2.set('value', start2 + pos2 * (end2-start2)/stepsize)
           
           pos3 = Math.round((val3-old_start1)/(old_end1-old_start1)*stepsize)
           slide3.set('value', start1 + pos3 * (end1-start1)/stepsize)
           
           pos4 = Math.round((val4-old_start2)/(old_end2-old_start2)*stepsize)
           slide4.set('value', start2 + pos4 * (end2-start2)/stepsize)
           
           
           
           
           
           
           ///////////////////////////////////////////
                      
           
           //change slider values
           slide.set("start", start1)
           slide.set("end", end1)
           slide.set("step", (end1 -start1)/stepsize)
           //slide.set("title", val1)
                      
           slide2.set("start", start2)
           slide2.set("end", end2)
           slide2.set("step", (end2 -start2)/stepsize)
           //slide2.set("title", val2)
           
           slide3.set("start", start1)
           slide3.set("end", end1)
           slide3.set("step", (end1 -start1)/stepsize)
           //slide3.set("title", val1+"end")
           
           slide4.set("start", start2)
           slide4.set("end", end2)
           slide4.set("step", (end2 -start2)/stepsize)
           //slide4.set("title", val2+"end")          
                     
           
           source.trigger('change');
                      
           // set axis labels
           x_axis.axis_label = value1;
           y_axis.axis_label = value2;
       
           
           '''


#cols = ["beta12", "beta23"]

source = ColumnDataSource(data=dict( x=val_a['RHill12'], y=val_a['RHill23'], 
                                    label = labels, RH12=val_a['RHill12'] )    )
#any values to be displayed in the hover tool need to be passed indivually. 
original_source = ColumnDataSource(data=val_a.to_dict(orient='list'))

TOOLS = [HoverTool(tooltips = [("RH", "@RH12")])]
#TOOLS = [ HoverTool(tooltips= [(c, '@' + c) for c in val_a.columns] + [('index', '$index')])]
# hover.tooltips.append(('index', '$index'))

#plot the figures
plot = figure(plot_width=650, plot_height=650, webgl = True, tools = TOOLS)#shuold speed up
#plot = figure(plot_width=650, plot_height=650)# ,   tools= TOOLS)

plot.scatter(x= "x",y="y", source=source, line_width=2, line_alpha=0.6, color = colors,
            legend = 'label', size = 3, name = "main")

slider = Slider(title ="Slider X Start", start=df_p['RHill12'].min(), end=df_p['RHill12'].max(),
                step=(df_p['RHill12'].max() - df_p['RHill12'].min()) /10., 
               value = df_p['RHill12'].min())
slider2 = Slider(title ="Slider Y Start", start=df_p['RHill23'].min(), end=df_p['RHill23'].max(),
                 step=(df_p['RHill23'].max() - df_p['RHill23'].min())/10., 
                 value = df_p['RHill23'].min())


slider3 = Slider(title ="Slider X End", start=df_p['RHill12'].min(), end=df_p['RHill12'].max(),
                step=(df_p['RHill12'].max() - df_p['RHill12'].min()) /10., 
               value = df_p['RHill12'].max())
slider4 = Slider(title ="Slider Y End", start=df_p['RHill23'].min(), end=df_p['RHill23'].max(),
                 step=(df_p['RHill23'].max() - df_p['RHill23'].min())/10., 
                 value = df_p['RHill23'].max())


callback = CustomJS(args=dict(source=source, original_source = original_source,
                              x_axis=plot.xaxis[0],y_axis=plot.yaxis[0], slide = slider ,
                              slide2 = slider2, slider3 =slider3,
                              slider4 = slider4), code=code2)



#create a separate callback to manage chaning data
callbackDRange = CustomJS(args = dict(source = source,original_source = original_source,
                              x_axis=plot.xaxis[0],y_axis=plot.yaxis[0], slide = slider ,
                              slide2 = slider2, slider3 =slider3,
                              slider4 = slider4), code=code )

slider.callback = callbackDRange
callback.args["slide"] = slider
callbackDRange.args["slide"] = slider


slider2.callback = callbackDRange
callback.args["slide2"] = slider2
callbackDRange.args["slide2"] = slider2

slider3.callback = callbackDRange
callback.args["slide3"] = slider3
callbackDRange.args["slide3"] = slider3

slider4.callback = callbackDRange
callback.args["slide4"] = slider4
callbackDRange.args["slide4"] = slider4


#Create two select widgets to pick the features of interest 
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="RHill12", callback = callback)
callback.args["val1"] = x_axis
callbackDRange.args["val1"]= x_axis

y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="RHill23", callback = callback)
callback.args["val2"] = y_axis
callbackDRange.args["val2"]= y_axis

#y_axis.js_on_change("value", callbackDRange)
#x_axis.js_on_change("value", callbackDRange)

y_axis.js_on_change("value", callback)
x_axis.js_on_change("value", callback)



#try this in a separate callback



plot.xaxis[0].axis_label = 'RHill12'
plot.yaxis[0].axis_label = 'RHill23'
#Error message due to legend, need to find a better way to do so

/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [6]:
#Display the graph in a jupyter notebook
layout = column(plot, x_axis, y_axis, row(slider, slider2),row(slider3, slider4) )
#show(widgetbox(button_1, slider, button_group, select, button_2, width=300))
show(layout, notebook_handle=True)

In [13]:
#output_file("WorkingExample2_lessdata.html", title ="Working Example ")
#show(layout)
#html = file_html(plot, CDN, "my plot")

In [42]:
#print type(html)

<type 'str'>


In [43]:
#doc = open("Working.html", "w")

#doc.write(html)
#doc.close()

In [44]:
#doc = open("Working.html", "r")
#print doc.read()


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>my plot</title>
        
<link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-0.12.5.min.css" type="text/css" />
        
<script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-0.12.5.min.js"></script>
<script type="text/javascript">
    Bokeh.set_log_level("info");
</script>
        <style>
          html {
            width: 100%;
            height: 100%;
          }
          body {
            width: 90%;
            height: 100%;
            margin: auto;
          }
        </style>
    </head>
    <body>
        
        <div class="bk-root">
            <div class="bk-plotdiv" id="45ee0aae-8bdd-4830-bf09-720cbb52532c"></div>
        </div>
        
        <script type="text/javascript">
            (function() {
          var fn = function() {
            Bokeh.safely(function() {
              var docs_json = {"e1bb109f-34ff-4a5e-93c7-fc906ea4